In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt

In [2]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [5]:
url="https://github.com/MinCiencia/Datos-COVID19/tree/master/output/"

In [6]:
request=requests.get(url)

In [7]:
root=url.split('github.com')[1]

In [8]:
prefix=url.split('/Min')[0]

In [9]:
folders=request.text.split(root)

In [10]:
#product folders
for idx in range(len(folders)):
    f=folders[idx]
    f=f.split('">')[0]
    folders[idx]=f
folders=[prefix+root+f for f in folders if 'producto' in f]

In [11]:
tables={}
for f in folders:
    files_folder=f.rsplit('/')[-1]
    files_folder_prefix='/MinCiencia/Datos-COVID19/blob/master/output/'+files_folder+'/'
    tables[files_folder]={}
    raw_path='https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/'+files_folder+'/'
    req=requests.get(f)
    files=req.text.split(files_folder_prefix)
    for idx in range(len(files)):
        files[idx]=files[idx].split('">')[0]
    files=[raw_path+f for f in files if '.csv' in f]
    for fl in files:
        try:
            print('reading file: '+fl)
            table=pd.read_csv(fl)
            table_name='_'.join(fl.rsplit('/',2)[-2:]).strip('.csv')
            tables[files_folder][table_name]=table
        except Exception as e:
            print(str(e))
            pass

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19_T.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario_T.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto13/CasosNuevosCumulativo.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto13/CasosNuevosCumulativo_T.csv
reading file: https://raw.githubusercontent.com/Min

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto2/2020-07-24-CasosConfirmados.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto2/2020-07-27-CasosConfirmados.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto2/2020-07-31-CasosConfirmados.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto2/2020-08-03-CasosConfirmados.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto2/2020-08-07-CasosConfirmados.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto2/2020-08-10-CasosConfirmados.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto2/2020-08-14-CasosConfirmados.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto20

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto31/Nacimientos_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto32/2020-Defunciones.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto32/2020-Defunciones_T.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto32/2020-Defunciones_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto32/Defunciones.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto32/Defunciones_T.csv


/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252

...

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-05-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-06-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-07-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-08-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-09-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-10-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-11-CasosConfirmados-totalRegional.csv

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-06-02-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-06-03-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-06-04-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-06-05-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-06-06-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-06-07-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-06-08-CasosConfirmados-totalRegional.csv

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-07-30-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-07-31-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-08-01-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-08-02-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-08-03-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-08-04-CasosConfirmados-totalRegional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-08-05-CasosConfirmados-totalRegional.csv

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto43/NO2-2020_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto43/O3-2010_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto43/O3-2011_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto43/O3-2012_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto43/O3-2013_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto43/O3-2014_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto43/O3-2015_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto43/O3-2016_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto52/Camas_UCI.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto52/Camas_UCI_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto52/Camas_UCI_t.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto53/confirmados_nacional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto53/confirmados_provinciales.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto53/confirmados_regionales.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto54/r.nacional.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto54/r.provincial.csv
reading file: https://raw.githubusercontent.com/MinCien

In [12]:
ortables=copy.deepcopy(tables)

In [13]:
def transpose(DF,date_col='Region',header_row='Comuna'):
    df=DF.copy()
    if date_col in df.columns:
        if header_row in list(df[date_col].unique()):
            df.columns=df[df[date_col]==header_row].iloc[0]
            df['Fecha']=pd.to_datetime(df[header_row],errors='coerce')
        else:
            df['Fecha']=pd.to_datetime(df[date_col],errors='coerce')
            df=df.dropna(subset=['Fecha'])
            #df['Fecha']=pd.Timestamp(df['Fecha'])
            df=df.drop(date_col,axis=1)
            
    df=df.loc[:,~df.columns.duplicated()]
    for c in df.columns:
        if c!= 'Fecha':
            df[c]=pd.to_numeric(df[c],errors='coerce')
    return df

In [14]:
def fix_datetime(df):
    for c in df.columns:
        if df[c].dtype=='<M8[ns]':
            print('converting datetime column to pydatetime for postrgres')
            df[c]=df[c].dt.to_pydatetime()
    return df

In [15]:
def to_diff(DF):
    v=[]
    columns=[ c for c in DF.columns if is_numeric_dtype(DF[c])]
    not_columns=[ c for c in DF.columns if not is_numeric_dtype(DF[c]) ]
    for c in columns:
        v.append(DF[c].diff())
        
    df=pd.DataFrame(v).T
    df=pd.concat([DF[not_columns],df],axis=1)
    df.iloc[0]=DF.iloc[0]    
        
    
    return df

In [16]:
def to_diff_std(df,group_col='Region',diff_col='Total'):
    df['Diff']=df.groupby(group_col)[diff_col].diff()
    df=df.fillna(0)
    df['LogTotal']=np.log(df['Total']+0.0001)
    df['LogNuevos']=np.log(df['Diff']+0.0001)
    df.loc[df['LogTotal']<0,'LogTotal']=0
    df.loc[df['LogNuevos']<0,'LogNuevos']=0
    df=df.fillna(0)
    
    slc=df[['LogTotal','LogNuevos']]
    slc=slc.replace(0,np.nan)
    slc=slc.dropna()
    X=slc['LogNuevos'].values.reshape(-1, 1)
    Y=slc['LogTotal'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    Y_pred = linear_regressor.predict(df['LogNuevos'].values.reshape(-1, 1))
    df['Regression']=Y_pred
    if 'Fecha' in df.columns:
        df['Fecha']=pd.to_datetime(df['Fecha'])
        df['Fecha']=df['Fecha'].dt.to_pydatetime()
    
    
    return df

In [17]:
for s in tables.keys():
    for n in tables[s].keys():
        #print(n)
        if '_T' in n:
            print('fixing transposed table: '+n)
            tables[s][n]=transpose(tables[s][n])

fixing transposed table: producto1_Covid-19_T
fixing transposed table: producto10_FallecidosEtario_T
fixing transposed table: producto13_CasosNuevosCumulativo_T
fixing transposed table: producto14_FallecidosCumulativo_T
fixing transposed table: producto15_FechaInicioSintomas_T
fixing transposed table: producto16_CasosGeneroEtario_T
fixing transposed table: producto17_PCREstablecimiento_T
fixing transposed table: producto18_TasaDeIncidencia
fixing transposed table: producto18_TasaDeIncidencia_T
fixing transposed table: producto18_TasaDeIncidencia_std
fixing transposed table: producto19_CasosActivosPorComuna_T
fixing transposed table: producto20_NumeroVentiladores_T
fixing transposed table: producto21_SintomasCasosConfirmados_T
fixing transposed table: producto21_SintomasHospitalizados_T
fixing transposed table: producto21_Sintomas_T
fixing transposed table: producto22_HospitalizadosEtario_Acumulado_T
fixing transposed table: producto22_HospitalizadosUCI_Acumulado_T
fixing transposed tab

In [18]:
for k in tables.keys():
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema producto1 exists, will not be created
schema producto10 exists, will not be created
schema producto11 exists, will not be created
schema producto12 exists, will not be created
schema producto13 exists, will not be created
schema producto14 exists, will not be created
schema producto15 exists, will not be created
schema producto16 exists, will not be created
schema producto17 exists, will not be created
schema producto18 exists, will not be created
schema producto19 exists, will not be created
schema producto2 exists, will not be created
schema producto20 exists, will not be created
schema producto21 exists, will not be created
schema producto22 exists, will not be created
schema producto23 exists, will not be created
schema producto24 exists, will not be created
schema producto25 exists, will not be created
schema producto26 exists, will not be created
schema producto27 exists, will not be created
schema producto28 exists, will not be created
schema producto29 exists, will not b

In [18]:
for schema in tables.keys():
    for name in tables[schema].keys():
        try:
            df=tables[schema][name]
            name=name.replace('-','_')
            print("creating table "+name+' ,schema: '+schema)
            df.to_sql(name, schema=schema,con=cnx,if_exists='replace')
            print("saving table"+path+name+timestamp+'.csv in cache')
            df.to_csv(path+name+timestamp+'.csv',encoding='utf-8')
        except Exception as e: 
            print(str(e))
            pass

creating table producto1_Covid_19 ,schema: producto1
saving table/data/ETLcache/producto1_Covid_19_19082020_063125.csv in cache
creating table producto1_Covid_19_T ,schema: producto1
saving table/data/ETLcache/producto1_Covid_19_T_19082020_063125.csv in cache
creating table producto1_Covid_19_std ,schema: producto1
saving table/data/ETLcache/producto1_Covid_19_std_19082020_063125.csv in cache
creating table producto10_FallecidosEtario ,schema: producto10
saving table/data/ETLcache/producto10_FallecidosEtario_19082020_063125.csv in cache
creating table producto10_FallecidosEtario_T ,schema: producto10
saving table/data/ETLcache/producto10_FallecidosEtario_T_19082020_063125.csv in cache
creating table producto10_FallecidosEtario_std ,schema: producto10
saving table/data/ETLcache/producto10_FallecidosEtario_std_19082020_063125.csv in cache
creating table producto13_CasosNuevosCumulativo ,schema: producto13
saving table/data/ETLcache/producto13_CasosNuevosCumulativo_19082020_063125.csv in 

saving table/data/ETLcache/producto2_2020_06_05_CasosConfirmado_19082020_063125.csv in cache
creating table producto2_2020_06_08_CasosConfirmado ,schema: producto2
saving table/data/ETLcache/producto2_2020_06_08_CasosConfirmado_19082020_063125.csv in cache
creating table producto2_2020_06_12_CasosConfirmado ,schema: producto2
saving table/data/ETLcache/producto2_2020_06_12_CasosConfirmado_19082020_063125.csv in cache
creating table producto2_2020_06_15_CasosConfirmado ,schema: producto2
saving table/data/ETLcache/producto2_2020_06_15_CasosConfirmado_19082020_063125.csv in cache
creating table producto2_2020_06_19_CasosConfirmado ,schema: producto2
saving table/data/ETLcache/producto2_2020_06_19_CasosConfirmado_19082020_063125.csv in cache
creating table producto2_2020_06_23_CasosConfirmado ,schema: producto2
saving table/data/ETLcache/producto2_2020_06_23_CasosConfirmado_19082020_063125.csv in cache
creating table producto2_2020_06_28_CasosConfirmado ,schema: producto2
saving table/dat

saving table/data/ETLcache/producto27_CasosNuevosSinSintomas_std_19082020_063125.csv in cache
creating table producto28_FechaInicioSintomas_reportadosSEREMI ,schema: producto28
saving table/data/ETLcache/producto28_FechaInicioSintomas_reportadosSEREMI_19082020_063125.csv in cache
creating table producto28_FechaInicioSintomas_reportadosSEREMIHistorico ,schema: producto28
saving table/data/ETLcache/producto28_FechaInicioSintomas_reportadosSEREMIHistorico_19082020_063125.csv in cache
creating table producto28_FechaInicioSintomas_reportadosSEREMIHistorico_std ,schema: producto28
saving table/data/ETLcache/producto28_FechaInicioSintomas_reportadosSEREMIHistorico_std_19082020_063125.csv in cache
creating table producto28_FechaInicioSintomas_reportadosSEREMI_T ,schema: producto28
saving table/data/ETLcache/producto28_FechaInicioSintomas_reportadosSEREMI_T_19082020_063125.csv in cache
creating table producto28_FechaInicioSintomas_reportadosSEREMI_std ,schema: producto28
saving table/data/ETLca

saving table/data/ETLcache/producto31_Nacimientos_T_19082020_063125.csv in cache
creating table producto31_Nacimientos_std ,schema: producto31
saving table/data/ETLcache/producto31_Nacimientos_std_19082020_063125.csv in cache
creating table producto32_2020_Defuncione ,schema: producto32
saving table/data/ETLcache/producto32_2020_Defuncione_19082020_063125.csv in cache
creating table producto32_2020_Defunciones_T ,schema: producto32
saving table/data/ETLcache/producto32_2020_Defunciones_T_19082020_063125.csv in cache
creating table producto32_2020_Defunciones_std ,schema: producto32
saving table/data/ETLcache/producto32_2020_Defunciones_std_19082020_063125.csv in cache
creating table producto32_Defuncione ,schema: producto32
(psycopg2.errors.TooManyColumns) tables can have at most 1600 columns

[SQL: 
CREATE TABLE producto32."producto32_Defuncione" (
	index BIGINT, 
	"Region" TEXT, 
	"Codigo region" BIGINT, 
	"Comuna" TEXT, 
	"Codigo comuna" BIGINT, 
	"2010-01-01" BIGINT, 
	"2010-01-02"

saving table/data/ETLcache/producto32_Defunciones_T_19082020_063125.csv in cache
creating table producto32_Defunciones_std ,schema: producto32
saving table/data/ETLcache/producto32_Defunciones_std_19082020_063125.csv in cache
creating table producto33_IndiceDeMovilidad_IM ,schema: producto33
saving table/data/ETLcache/producto33_IndiceDeMovilidad_IM_19082020_063125.csv in cache
creating table producto33_IndiceDeMovilidad_IM_T ,schema: producto33
saving table/data/ETLcache/producto33_IndiceDeMovilidad_IM_T_19082020_063125.csv in cache
creating table producto33_IndiceDeMovilidad_IM_externo ,schema: producto33
saving table/data/ETLcache/producto33_IndiceDeMovilidad_IM_externo_19082020_063125.csv in cache
creating table producto33_IndiceDeMovilidad_IM_externo_T ,schema: producto33
saving table/data/ETLcache/producto33_IndiceDeMovilidad_IM_externo_T_19082020_063125.csv in cache
creating table producto33_IndiceDeMovilidad_IM_interno ,schema: producto33
saving table/data/ETLcache/producto33_I

...

(psycopg2.ProgrammingError) incomplete placeholder: '%(' without ')'
[SQL: INSERT INTO producto4."producto4_2020_06_15_CasosConfirmados_totalRegional" (index, "Region", "Casos totales acumulados", "Casos nuevos totales", "Casos nuevos con sintomas", "Casos nuevos sin sintomas*", "Fallecidos totales", "%% Total") VALUES (%(index)s, %(Region)s, %(Casos totales acumulados)s, %(Casos nuevos totales)s, %(Casos nuevos con sintomas)s, %(Casos nuevos sin sintomas*)s, %(Fallecidos totales)s, %(% Total)s)]
[parameters: ({'index': 0, 'Region': 'Arica y Parinacota', 'Casos totales acumulados': 1139, 'Casos nuevos totales': 30, 'Casos nuevos con sintomas': 30, 'Casos nuevos sin sintomas*': 0, 'Fallecidos totales': 11, '% Total': '0.63%'}, {'index': 1, 'Region': 'Tarapacá', 'Casos totales acumulados': 4258, 'Casos nuevos totales': 113, 'Casos nuevos con sintomas': 90, 'Casos nuevos sin sintomas*': 23, 'Fallecidos totales': 58, '% Total': '2.37%'}, {'index': 2, 'Region': 'Antofagasta', 'Casos totales

saving table/data/ETLcache/producto4_2020_06_21_CasosConfirmados_totalRegional_19082020_063125.csv in cache
creating table producto4_2020_06_22_CasosConfirmados_totalRegional ,schema: producto4
saving table/data/ETLcache/producto4_2020_06_22_CasosConfirmados_totalRegional_19082020_063125.csv in cache
creating table producto4_2020_06_23_CasosConfirmados_totalRegional ,schema: producto4
saving table/data/ETLcache/producto4_2020_06_23_CasosConfirmados_totalRegional_19082020_063125.csv in cache
creating table producto4_2020_06_24_CasosConfirmados_totalRegional ,schema: producto4
saving table/data/ETLcache/producto4_2020_06_24_CasosConfirmados_totalRegional_19082020_063125.csv in cache
creating table producto4_2020_06_25_CasosConfirmados_totalRegional ,schema: producto4
saving table/data/ETLcache/producto4_2020_06_25_CasosConfirmados_totalRegional_19082020_063125.csv in cache
creating table producto4_2020_06_26_CasosConfirmados_totalRegional ,schema: producto4
saving table/data/ETLcache/pro

saving table/data/ETLcache/producto4_2020_08_04_CasosConfirmados_totalRegional_19082020_063125.csv in cache
creating table producto4_2020_08_05_CasosConfirmados_totalRegional ,schema: producto4
saving table/data/ETLcache/producto4_2020_08_05_CasosConfirmados_totalRegional_19082020_063125.csv in cache
creating table producto4_2020_08_06_CasosConfirmados_totalRegional ,schema: producto4
saving table/data/ETLcache/producto4_2020_08_06_CasosConfirmados_totalRegional_19082020_063125.csv in cache
creating table producto4_2020_08_07_CasosConfirmados_totalRegional ,schema: producto4
saving table/data/ETLcache/producto4_2020_08_07_CasosConfirmados_totalRegional_19082020_063125.csv in cache
creating table producto4_2020_08_08_CasosConfirmados_totalRegional ,schema: producto4
saving table/data/ETLcache/producto4_2020_08_08_CasosConfirmados_totalRegional_19082020_063125.csv in cache
creating table producto4_2020_08_09_CasosConfirmados_totalRegional ,schema: producto4
saving table/data/ETLcache/pro

saving table/data/ETLcache/producto43_NO2_2011_std_19082020_063125.csv in cache
creating table producto43_NO2_2012_std ,schema: producto43
saving table/data/ETLcache/producto43_NO2_2012_std_19082020_063125.csv in cache
creating table producto43_NO2_2013_std ,schema: producto43
saving table/data/ETLcache/producto43_NO2_2013_std_19082020_063125.csv in cache
creating table producto43_NO2_2014_std ,schema: producto43
saving table/data/ETLcache/producto43_NO2_2014_std_19082020_063125.csv in cache
creating table producto43_NO2_2015_std ,schema: producto43
saving table/data/ETLcache/producto43_NO2_2015_std_19082020_063125.csv in cache
creating table producto43_NO2_2016_std ,schema: producto43
saving table/data/ETLcache/producto43_NO2_2016_std_19082020_063125.csv in cache
creating table producto43_NO2_2017_std ,schema: producto43
saving table/data/ETLcache/producto43_NO2_2017_std_19082020_063125.csv in cache
creating table producto43_NO2_2018_std ,schema: producto43
saving table/data/ETLcache/

saving table/data/ETLcache/producto47_MediaMovil_std_19082020_063125.csv in cache
creating table producto48_SOCHIMI ,schema: producto48
saving table/data/ETLcache/producto48_SOCHIMI_19082020_063125.csv in cache
creating table producto48_SOCHIMI_T ,schema: producto48
saving table/data/ETLcache/producto48_SOCHIMI_T_19082020_063125.csv in cache
creating table producto48_SOCHIMI_std ,schema: producto48
saving table/data/ETLcache/producto48_SOCHIMI_std_19082020_063125.csv in cache
creating table producto49_Positividad_Diaria_Media ,schema: producto49
saving table/data/ETLcache/producto49_Positividad_Diaria_Media_19082020_063125.csv in cache
creating table producto49_Positividad_Diaria_Media_T ,schema: producto49
saving table/data/ETLcache/producto49_Positividad_Diaria_Media_T_19082020_063125.csv in cache
creating table producto49_Positividad_Diaria_Media_std ,schema: producto49
saving table/data/ETLcache/producto49_Positividad_Diaria_Media_std_19082020_063125.csv in cache
creating table pro

In [19]:
derived_tables={}
for s in tables.keys():
    for n in tables[s].keys():
        #print(n)
        if 'cumulativo' in n.lower():
            print('generating diff table for cumulative table: '+n)
            nn=n.lower().replace('cumulativo','_diff_')
            if s not in derived_tables.keys():
                derived_tables[s]={}
            print('new name: '+nn)
            if 'std' not in nn:
                derived_tables[s][nn]=to_diff(tables[s][n])
            else:
                print('standard form with linear regression')
                derived_tables[s][nn]=to_diff_std(tables[s][n])
                derived_tables[s][nn]['Fecha']=derived_tables[s][nn]['Fecha'].dt.to_pydatetime()
        

generating diff table for cumulative table: producto13_CasosNuevosCumulativo
new name: producto13_casosnuevos_diff_
generating diff table for cumulative table: producto13_CasosNuevosCumulativo_T
new name: producto13_casosnuevos_diff__t
generating diff table for cumulative table: producto13_CasosNuevosCumulativo_std
new name: producto13_casosnuevos_diff__std
standard form with linear regression
generating diff table for cumulative table: producto14_FallecidosCumulativo
new name: producto14_fallecidos_diff_
generating diff table for cumulative table: producto14_FallecidosCumulativo_T
new name: producto14_fallecidos_diff__t
generating diff table for cumulative table: producto14_FallecidosCumulativo_std
new name: producto14_fallecidos_diff__std
standard form with linear regression


/opt/conda/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


generating diff table for cumulative table: producto3_CasosTotalesCumulativo
new name: producto3_casostotales_diff_
generating diff table for cumulative table: producto3_CasosTotalesCumulativo_T
new name: producto3_casostotales_diff__t
generating diff table for cumulative table: producto3_CasosTotalesCumulativo_std
new name: producto3_casostotales_diff__std
standard form with linear regression


In [20]:
for schema in derived_tables.keys():
    for name in derived_tables[schema].keys():
        try:
            df=derived_tables[schema][name]
            name=name.replace('-','_')
            print("creating table "+name+' ,schema: '+schema)
            df.to_sql(name, schema=schema,con=cnx,if_exists='replace')
            print("saving table"+path+name+timestamp+'.csv in cache')
            df.to_csv(path+name+timestamp+'.csv',encoding='utf-8')
        except Exception as e: 
            print(str(e))
            pass

creating table producto13_casosnuevos_diff_ ,schema: producto13
saving table/data/ETLcache/producto13_casosnuevos_diff__19082020_063125.csv in cache
creating table producto13_casosnuevos_diff__t ,schema: producto13
saving table/data/ETLcache/producto13_casosnuevos_diff__t_19082020_063125.csv in cache
creating table producto13_casosnuevos_diff__std ,schema: producto13
saving table/data/ETLcache/producto13_casosnuevos_diff__std_19082020_063125.csv in cache
creating table producto14_fallecidos_diff_ ,schema: producto14
saving table/data/ETLcache/producto14_fallecidos_diff__19082020_063125.csv in cache
creating table producto14_fallecidos_diff__t ,schema: producto14
saving table/data/ETLcache/producto14_fallecidos_diff__t_19082020_063125.csv in cache
creating table producto14_fallecidos_diff__std ,schema: producto14
saving table/data/ETLcache/producto14_fallecidos_diff__std_19082020_063125.csv in cache
creating table producto3_casostotales_diff_ ,schema: producto3
saving table/data/ETLcach

In [23]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/covid-chile.ipynb')

0